$$ -D_1 (\frac{d^2\phi_1(x,y)}{dx^2} + \frac{d^2\phi_1(x,y)}{dy^2}) + \Sigma_{r,1} \phi_1((x,y)) = \frac{1}{k_{eff}} (\nu \Sigma_{f,1} \phi_1(x,y)+\nu \Sigma_{f,2} \phi_2(x,y)) $$
e
$$ -D_2 (\frac{d^2\phi_2(x,y)}{dx^2} + \frac{d^2\phi_2(x,y)}{dy^2}) + \Sigma_{r,2} \phi_2(x,y) = \Sigma_{21} \phi_1(x,y) $$

$$$$

$$
- \frac{D_1}{dx^2} (\phi_{1,i-1,j} - 2\phi_{1,i,j} + \phi_{1,i+1,j}) - \frac{D_1}{dy^2} (\phi_{1,i,j-1} - 2\phi_{1,i,j} + \phi_{1,i,j+1}) + (\Sigma_{a1} + \Sigma_{12}) \phi_{1,i,j} =$$
$$\frac{1}{k_{eff}}(\nu_1\Sigma_{f1} \phi_{1,i,j} + \nu_2\Sigma_{f2} \phi_{2,i,j})
$$

$$$$

$$
- \frac{D_2}{dx^2} (\phi_{2,i-1,j} - 2\phi_{2,i,j} + \phi_{2,i+1,j}) - \frac{D_2}{dy^2} (\phi_{2,i,j-1} - 2\phi_{2,i,j} + \phi_{2,i,j+1}) + (\Sigma_{a2} + \Sigma_{21}) \phi_{2,i,j} = \Sigma_{12} \phi_{1,i,j}
$$

$$$$


In [ ]:
#######################################
### PROBLEMA HOMOGÊNEO PARA 2G E 2D ###
#######################################

import time
import cupy as cp
Criterio_k_eff = 1e-07  # Critério de convergência para o fator de multiplicação

# Parâmetros do Grupo 1
D1 = 1.5000
Sigma_a1 = 0.0130
nu_Sigma_f1 = 0.0065
Sigma_12 = 0.0200
Sigma_r1 = Sigma_a1 + Sigma_12

# Parâmetros do Grupo 2
D2 = 0.4000
Sigma_a2 = 0.1800
nu_Sigma_f2 = 0.2400
Sigma_21 = 0.0000
Sigma_r2 = Sigma_a2 + Sigma_21

# Números de malhas do problema
N = 50

# Dimensão do problema
L = 50.0

dx = L/N      # Dimensão de cada lateral da malha em x
dy = L/N      # Dimensão de cada lateral da malha em y

start_time_cp = time.time()  # Pega o tempo inicial dos cálculos


# Inicializa as matrizes de discretização
A1 = cp.zeros(((N-1)**2,(N-1)**2))
A2 = cp.zeros(((N-1)**2,(N-1)**2))
A3 = cp.zeros(((N-1)**2,(N-1)**2))
A0 = cp.zeros(((N-1)**2,(N-1)**2))
F1 = cp.zeros(((N-1)**2,(N-1)**2))
F2 = cp.zeros(((N-1)**2,(N-1)**2))
F0 = cp.zeros(((N-1)**2,(N-1)**2))

for j in range(+1,N-1 +1):
    for i in range(1,N-1 +1):
        n = (N-1) * (j - 1)+ i;
        n1 = (N-1) * (j-1 - 1) + i;
        n2 = (N-1) * (j-1 + 1) + i;

        if n1 >= 1:                  # fluxo i, j-1
            A1[n -1,n1 -1] = - D1/dy**2
            A2[n -1,n1 -1] = - D2/dy**2

        if n2 <= (N - 1)**2:         # fluxo i, j+1
            A1[n -1,n2 -1] = - D1/dy**2
            A2[n -1,n2 -1] = - D2/dy**2


        if n > 1:                    # fluxo i-1, j
            if i == 1:
                A1[n -1,n-1 -1] = 0
                A2[n -1,n-1 -1] = 0
            else:
                A1[n -1,n-1 -1] = - D1/dx**2
                A2[n -1,n-1 -1] = - D2/dx**2

        if n < (N - 1)**2:           # fluxo i+1, j
            if i == (N - 1):
                A1[n -1,n+1 -1] = 0
                A2[n -1,n+1 -1] = 0
            else:
                A1[n -1,n+1 -1] = - D1/dx**2
                A2[n -1,n+1 -1] = - D2/dx**2


        # fluxo i,j
        A1[n -1,n -1] = 2*D1 * (1/dy**2 + 1/dx**2) + Sigma_r1
        A2[n -1,n -1] = 2*D2 * (1/dy**2 + 1/dx**2) + Sigma_r2
        A3[n -1,n -1] = -Sigma_12
        F1[n -1,n -1] = nu_Sigma_f1
        F2[n -1,n -1] = nu_Sigma_f2


A = cp.vstack([cp.hstack([A1, A0]), cp.hstack([A3, A2])])
F = cp.vstack([cp.hstack([F1, F2]), cp.hstack([F0, F0])])


k_eff = 1.0                              # Inicializa o fator de multiplicação
Numero_Iteracoes = 0                     # Inicializa o número de iterações
k_eff_old = k_eff                        # Define o fator de multiplicação anterior

# Inicializa o fluxo
phi = cp.ones((N-1)**2*2)

Fonte_old = (1 / k_eff) * cp.dot(F,phi)  # Inicializa o termo de fonte
Fonte_Somada_old = cp.sum(Fonte_old)     # Inicializa a fonte somadaa

# Inicia as iterações
while True:
    Numero_Iteracoes += 1    # Acumula o número de iterações

    # Resolve o sistema linear
    phi = cp.linalg.solve(A,Fonte_old)

    # Calcula a nova Fonte
    Fonte = (1 / k_eff) * cp.dot(F,phi)

    # Calcula a nova Fonte somada
    Fonte_Somada = cp.sum(Fonte)

    # Calcula o novo k_eff
    k_eff = k_eff_old * Fonte_Somada / Fonte_Somada_old

    # Verifica a convergência
    if abs(k_eff - k_eff_old) < Criterio_k_eff:
        break
    else:
        k_eff_old = k_eff
        Fonte_old = Fonte
        Fonte_Somada_old = Fonte_Somada


end_time_cp = time.time() # Pega o tempo final dos cálculos

# Calcule os tempos de execução
time_cp = end_time_cp - start_time_cp
print(f"Tempo de execução com NumPy: {time_cp:.4f} segundos")
print("Numero de iterações:", Numero_Iteracoes)
print("Fator de multiplicação:", k_eff)


psi = cp.abs(phi[:])
psi = psi/cp.max(psi)


phi1 = cp.zeros((N-1,N-1))
phi2 = cp.zeros((N-1,N-1))
psi1 = psi[0:(N-1)**2]
psi2 = psi[(N-1)**2:]

for j in range(+1,N-1 +1):
    for i in range(+1,N-1 +1):
        n = (N-1) * (j-1)+ i;
        phi1[i -1,j -1] = psi1[n -1]
        phi2[i -1,j -1] = psi2[n -1]

print("")
print("Fluxo Grupo 1:")
print(phi1)
print("")
print("Fluxo Grupo 2:")
print(phi2)
